# Program Collections generator

This script parses a xlsx file containing metadata about Program Collections and searches the NIAID Data Ecosystem Discovery Portal for records that should be included as part of each program. It then generates the corrections files needed to add the program collection to the Discovery Portal

In [2]:
import os
import requests
import json
import pandas as pd

In [3]:
script_path = os.getcwd()
data_path = os.path.join(script_path,'data')
file_path = os.path.join(data_path,'Program Collections.xlsx')
p1_path = os.path.abspath(os.path.join(script_path, os.pardir))
parent_path = os.path.abspath(os.path.join(p1_path, os.pardir))
correction_path = os.path.join(parent_path,'nde-metadata-corrections','collections_corrections')
print(correction_path)

C:\Users\gtsueng\Anaconda3\envs\nde\nde-metadata-corrections\collections_corrections


In [4]:
def parse_array_text(arraytext):
    if ',' in arraytext:
        arrayinfo = arraytext.split(',')
        cleanarray = [x.strip() for x in arrayinfo]
    elif '|' in arraytext:
        arrayinfo = arraytext.split('|')
        cleanarray = [x.strip() for x in arrayinfo]
    else:
        cleanarray = arraytext
    return cleanarray

def generate_meta_file(correction_path, row):
    clean_descript = f"{row['description']} For more information, visit the NIAID program page: {row['niaidURL']}"
    altname = parse_array_text(row['alternateName'])
    parentorgs = parse_array_text(row['parentOrganization']) 
    meta_dict = {"@type": "ResearchProject", "name":row["name"],
                 "description":clean_descript, "alternateName":altname,
                 "url": row["url"], "parentOrganization":parentorgs} 
    file_name = f'{row["fileName"]}_correction.json'
    output_dict = {"sourceOrganization":[meta_dict]}
    with open(os.path.join(correction_path,file_name),'w') as outwrite:
        outwrite.write(json.dumps(output_dict,indent=4))

In [16]:
base_meta = pd.read_excel(file_path, 'metadata', engine='openpyxl')
test_meta = base_meta.loc[~base_meta['fundingIDList'].isna()]
print(test_meta.head(n=5))

     fileName                     name  \
0      SysBio    NIAID Systems Biology   
8   CRSTAL-ID  NIAID CRSTAL-ID program   
11       GCID       NIAID GCID Network   
16       HIPC       NIAID HIPC Network   

                                          description  \
0   The NIAID/Division of Microbiology and Infecti...   
8   The CRSTAL-ID program, formerly known as the S...   
11  The NIAID Genomic Centers for Infectious Disea...   
16  The Human Immunology Project Consortium (HIPC)...   

                                        alternateName parentOrganization  \
0                                                 NaN              NIAID   
8   CRSTAL-ID, Centers for Research on Structural ...              NIAID   
11      GCID, Genomic Centers for Infectious Diseases              NIAID   
16          HIPC, Human Immunology Project Consortium              NIAID   

                                                  url  \
0   https://www.niaid.nih.gov/research/systems-bio...   
8       

In [22]:
test_row = test_meta.iloc[1]
generate_meta_file(correction_path, test_row)

In [7]:
activity_codes_df = pd.read_csv(os.path.join(data_path,'NIH_IC_codes.tsv'),delimiter='\t',header=0)
print(activity_codes_df.head(n=2))

  Acronym                                          Full Name  Code
0      FIC               John E. Fogarty International Center   TW
1    NCATS  National Center for Advancing Translational Sc...   TR


In [19]:
def load_codes(data_path):
    activity_codes_df = pd.read_csv(os.path.join(data_path,'NIH_activity_codes.csv'),delimiter=',',header=0)
    tmp_codes = activity_codes_df['Activity Code'].unique().tolist()
    act_codes = [x.strip() for x in tmp_codes]
    ic_codes_df = pd.read_csv(os.path.join(data_path,'NIH_IC_codes.tsv'),delimiter='\t',header=0)
    temp_codes = ic_codes_df['Code'].unique().tolist()
    ic_codes = [x.strip() for x in temp_codes]
    return act_codes, ic_codes

def check_grantID_start(grantID, act_codes, ic_codes):
    first_char = grantID[0]
    try:
        int(first_char)
        idstart = "application_type"
    except:
        two_char = grantID[0:2]
        if '-' in grantID[0:3]:
            three_char = grantID[0:4].replace('-','')
        else:
            three_char = grantID[0:3]
        if two_char in ic_codes:
            idstart = "ic_code"
        elif three_char in act_codes:
            idstart = "activity_code"
        else:
            idstart = "unknown"
    return idstart

def check_grantID_end(grantID):
    if '-' in grantID[-3:-1]:
        idend = "FY"
    else:
        idend = "project_code"
    return idend

def parse_apptype_start(grantID): ## Eg. 1-R01 vs 1R01
    applTypeCode = str(grantID[0])
    if '-' in grantID[0:2]:
        remaining_grantID = grantID[2:]
    else:
        remaining_grantID = grantID[1:]
    return applTypeCode, remaining_grantID

def parse_actcode_start(grantID):
    if '-' in grantID[0:3]: ## Eg. U-01 vs U01 vs U-01- vs U01-
        activityCode = grantID[0:4].replace('-','')
        if '-' in grantID[4:7]:
            remaining_grantID = grantID[5:]
        else:
            remaining_grantID = grantID[4:]
    else:
        activityCode = grantID[0:3]
        if '-' in grantID[3:6]:
            remaining_grantID = grantID[4:]
        else:
            remaining_grantID = grantID[3:]
    return activityCode, remaining_grantID

def parse_iccode_start(grantID):
    icCode = grantID[0:2]
    if '-' in grantID[2:5]: ## Eg. AI089992-11, AI-089992-11
        remaining_grantID = grantID[3:]
    else:
        remaining_grantID = grantID[2:]
    return icCode, remaining_grantID

def parse_serial_start(grantID):
    if len(grantID) > 6:
        serialNum = grantID[0:6]
        remaining_grantID = grantID[7:]
    else:
        serialNum = grantID
        remaining_grantID = ''
    return serialNum, remaining_grantID

def parse_grantID(grantID, act_codes, ic_codes):
    idstart = check_grantID_start(grantID, act_codes, ic_codes)
    idend = check_grantID_end(grantID)
    if idend == "FY":
        supportYear = grantID[-2:]
    else:
        supportYear = "not found"
    if idstart == "application_type":
        applTypeCode, grantID_child0 = parse_apptype_start(grantID)
        activityCode, grantID_child1 = parse_actcode_start(grantID_child0)
        icCode, grantID_child2 = parse_iccode_start(grantID_child1)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)
    elif idstart == "activity_code":
        applTypeCode = "not found"
        activityCode, grantID_child1 = parse_actcode_start(grantID)
        icCode, grantID_child2 = parse_iccode_start(grantID_child1)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)
    elif idstart == "ic_code":
        applTypeCode = "not found"
        activityCode = "not found"
        icCode, grantID_child2 = parse_iccode_start(grantID)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)
    else:
        applTypeCode = "not found"
        activityCode = "not found"
        icCode = "not found"
        serialNum = "not found"
    grantObject = {"grantID":grantID,
                   "applTypeCode":applTypeCode, 
                   "activityCode":activityCode, 
                   "icCode":icCode, 
                   "serialNum":serialNum, 
                   "supportYear":supportYear}
    return grantObject

grantlist = parse_array_text(test_row['fundingIDList'])
print(grantlist[0])

AI089992-11


In [9]:
grantIDList = ["1-R01-AI073685-01","1R01-AI073685-01","1-R01AI073685-01","1-R-01AI073685-01","R01AI073685-01","R01AI073685-01","R01-AI073685-01","AI073685-01","1-R-01-AI073685-01", "AI073685"]
act_codes, ic_codes = load_codes(data_path)
for eachgrant in grantIDList:
    grantObject = parse_grantID(eachgrant, act_codes, ic_codes)
    print(grantObject)
    

{'grantID': '1-R01-AI073685-01', 'applTypeCode': '1', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': '1R01-AI073685-01', 'applTypeCode': '1', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': '1-R01AI073685-01', 'applTypeCode': '1', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': '1-R-01AI073685-01', 'applTypeCode': '1', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': 'R01AI073685-01', 'applTypeCode': 'not found', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': 'R01AI073685-01', 'applTypeCode': 'not found', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': 'R01-AI073685-01', 'applTypeCode': 'not found', 'activityCode': 'R01', 'icCode': 'AI', 'serialNum': '073685', 'supportYear': '01'}
{'grantID': 'AI073685-01', 

### Searching for records related to grant IDs:

To do:
- Pull the datasets associated with the differently formatted grantIDs
- Compare the results to identify the best approach for pulling records based on grantIDs
  - It looks like the combined icCode+project number may yield the most results

In [10]:
def search_for_records(grantlist):
    resultlist = []
    faillist = []
    for eachgrant in grantlist:
        r = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:*{eachgrant}*&fields=_id,funding.identifier&size=500")
        s = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:{eachgrant}&fields=_id,funding.identifier&size=500")
        temp = json.loads(r.text)
        temp2 = json.loads(s.text)
        if len(temp['hits'])>0:
            if len(temp2['hits'])>0:
                alltemp = temp['hits']+temp2['hits']
            else:
                alltemp = temp['hits']
        elif len(temp2['hits'])>0:
                alltemp = temp2['hits']
        else:
            faillist.append(eachgrant)
            alltemp = []
        if len(alltemp)>0:
            for eachhit in alltemp:
                tmpid = eachhit['_id']
                for eachfunding in eachhit['funding']:
                    if eachgrant in eachfunding['identifier']:
                        resultlist.append({"query":eachgrant,"_id":tmpid,"fundID":eachfunding['identifier']})
    resultdf = pd.DataFrame(resultlist)
    clean_result = resultdf.drop_duplicates(keep="first")
    return clean_result

In [11]:
grantIDList = ["1-R01-AI073685-01","1R01-AI073685-01","1-R01AI073685-01","1-R-01AI073685-01","R01AI073685","R01AI073685-01","R01 AI073685-01", "R01 AI07368501","R01-AI073685-01","AI073685-01","1-R-01-AI073685-01", "AI073685"]
clean_result = search_for_records(grantIDList)

In [12]:
print(clean_result)

             query            _id           fundID
0      R01AI073685  ds_61fc1af3a0  1R01AI073685-01
2      R01AI073685  ds_69b30c3f3f  1R01AI073685-01
3      R01AI073685  ds_e639f19429  1R01AI073685-01
5      R01AI073685  ds_5dc6e26d66  1R01AI073685-01
7      R01AI073685  ds_7fb8ffda8b  1R01AI073685-01
9      R01AI073685  ds_e3b3b46a00  1R01AI073685-01
10  R01AI073685-01  ds_61fc1af3a0  1R01AI073685-01
12  R01AI073685-01  ds_69b30c3f3f  1R01AI073685-01
13  R01AI073685-01  ds_e639f19429  1R01AI073685-01
15  R01AI073685-01  ds_5dc6e26d66  1R01AI073685-01
17  R01AI073685-01  ds_7fb8ffda8b  1R01AI073685-01
19  R01AI073685-01  ds_e3b3b46a00  1R01AI073685-01
20     AI073685-01  ds_61fc1af3a0  1R01AI073685-01
22     AI073685-01  ds_69b30c3f3f  1R01AI073685-01
23     AI073685-01  ds_e639f19429  1R01AI073685-01
25     AI073685-01  ds_5dc6e26d66  1R01AI073685-01
27     AI073685-01  ds_7fb8ffda8b  1R01AI073685-01
29     AI073685-01  ds_e3b3b46a00  1R01AI073685-01
30        AI073685  ds_61fc1af3

In [13]:
## parse the funder ids from the spreadsheet and format into icCode+project number
def parse_program_funding(fundingInfo):
    grantlist = []
    temp = fundingInfo.split("|")
    temp2 = [x.strip() for x in temp]
    temp3 = [x.replace("*","") for x in temp2]
    for eachgrant in temp3:
        try:
            grantObject = parse_grantID(eachgrant, act_codes, ic_codes)
            grantlist.append(grantObject['icCode']+grantObject['serialNum'])
        except:
            grantlist.append(eachgrant)
    return grantlist

In [14]:
def generate_inclusion_list(row):
    filename = f"{row['fileName']}_records.txt"
    fundingInfo = row['fundingIDList']
    grantlist = parse_program_funding(fundingInfo)
    clean_result = search_for_records(grantlist)
    print(clean_result)
    with open(os.path.join(correction_path,filename),'w') as outwrite:
        for eachrecord in clean_result['_id'].unique().tolist():
            outwrite.write(f'https://data.niaid.nih.gov/resources?id={eachrecord}\n')

In [24]:
#print(test_meta.iloc[1]['fundingIDList'])
generate_inclusion_list(test_meta.iloc[2])
#print(test_meta.head(n=2))
#test_meta.apply(lambda row: generate_inclusion_list(row), axis=1)

        query            _id           fundID
0    AI110818      gse118400  1U19AI110818-01
1    AI110818      gse114842  1U19AI110818-01
2    AI110818      gse129833  1U19AI110818-01
3    AI110818  ds_74a7480ae4  1U19AI110818-01
4    AI110818  ds_76ede52023  1U19AI110818-01
..        ...            ...              ...
182  AI144297      gse150918  1U19AI144297-01
183  AI144297  ds_3901f3433e  1U19AI144297-01
184  AI144297      gse196385  1U19AI144297-01
185  AI144297      phs002258  1U19AI144297-01
186  AI144297      gse227205  1U19AI144297-01

[165 rows x 3 columns]


In [66]:
eachgrant=grantIDList[-1]
r = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:*{eachgrant}*&fields=_id,funding.identifier&size=500")
temp = json.loads(r.text)
print(temp)

{'took': 42, 'total': 6, 'max_score': 2.0, 'hits': [{'_id': 'ds_61fc1af3a0', '_score': 2.0, 'funding': [{'identifier': '1R01AI080799-01A1'}, {'identifier': '1R01AI073685-01'}, {'identifier': '1R01AI078183-01A2'}, {'identifier': 'AI77680'}, {'identifier': '2R37AI029746-18'}, {'identifier': '1R01AI073745-01A2'}, {'identifier': '1R01AI095842-01'}, {'identifier': '1R01AI042361-01A1'}, {'identifier': '1R21AI099528-01A1'}, {'identifier': '1R01AI077680-01A2'}, {'identifier': '1R01AI080799-01A1'}, {'identifier': '1R21AI094289-01'}, {'identifier': '1R01AI073685-01'}, {'identifier': '1R01AI078183-01A2'}, {'identifier': '1R21AI094289-01'}, {'identifier': '1R56AI077680-01A1'}, {'identifier': '1R01AI095842-01'}, {'identifier': '1R21AI105575-01'}, {'identifier': '1R21AI105575-01'}, {'identifier': '1R21AI099528-01A1'}, {'identifier': '1R01AI073745-01A2'}, {'identifier': 'AI29746'}, {'identifier': '1R01AI042361-01A1'}, {'identifier': '1R01AI029746-01'}]}, {'_id': 'ds_69b30c3f3f', '_ignored': ['all.key